# Bring your own LLMs

Ragas uses langchain under the hood for connecting to LLMs for metrices that require them. This means you can swap out the default LLM we use (`gpt-3.5-turbo-16k`) to use any 100s of API supported out of the box with langchain.

- [Completion LLMs Supported](https://api.python.langchain.com/en/latest/api_reference.html#module-langchain.llms)
- [Chat based LLMs Supported](https://api.python.langchain.com/en/latest/api_reference.html#module-langchain.chat_models)

This guide will show you how to use another or LLM API for evaluation.

:::{Note}
If your looking to use Azure OpenAI for evaluation checkout [this guide](./azure-openai.ipynb)
:::

## Evaluating with GPT4

Ragas uses gpt3.5 by default but using gpt4 for evaluation can improve the results so lets use that for the `Faithfulness` metric

To start-off, we initialise the gpt4 `chat_model` from langchain

In [1]:
# make sure you have you OpenAI API key ready
import os

os.environ["OPENAI_API_KEY"] = "api-key"

In order to you the Langchain LLM you have to use the `RagasLLM` wrapper. This help the Ragas library specify the interfaces that will be used internally by the metrics and what is exposed via the Langchain library. You can also use other LLM APIs in tools like LlamaIndex and LiteLLM but creating your own implementation of `RagasLLM` that supports it.

In [ ]:
from ragas.llms import LangchainLLM



Substitute the llm in `Metric` instance with the newly create GPT4 model.

In [ ]:
from ragas.metrics import faithfulness

# faithfulness.llm = gpt4_wrapper

That's it! faithfulness will now be using GPT-4 under the hood for evaluations.

Now lets run the evaluations using the example from [quickstart](../quickstart.ipnb).

In [2]:
!pip install langchain datasets ragas  vllm -Uqq

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [23 lines of output]
      C:\Users\jaikr\AppData\Local\Temp\pip-build-env-63azlv3r\overlay\Lib\site-packages\torch\nn\modules\transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:84.)
        device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),
      Traceback (most recent call last):
        File "C:\Users\jaikr\.conda\envs\Jupyter_1\lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "C:\Users\jaikr\.conda\envs\Jupyter_1\lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
        File "C:

In [ ]:
# data
from datasets import load_dataset

fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")
fiqa_eval

Generating baseline split:   0%|          | 0/30 [00:00<?, ? examples/s]

DatasetDict({
    baseline: Dataset({
        features: ['question', 'ground_truths', 'answer', 'contexts'],
        num_rows: 30
    })
})

In [19]:
# # evaluate
# from ragas import evaluate

# result = evaluate(
#     fiqa_eval["baseline"].select(range(5)),  # showing only 5 for demonstration
#     metrics=[faithfulness],
# )

# result

## Evaluating with Open-Source LLMs

You can also use any of the Open-Source LLM for evaluating. Ragas support most the the deployment methods like [HuggingFace TGI](https://python.langchain.com/docs/integrations/llms/huggingface_textgen_inference), [Anyscale](https://python.langchain.com/docs/integrations/llms/anyscale), [vLLM](https://python.langchain.com/docs/integrations/llms/vllm) and many [more](https://python.langchain.com/docs/integrations/llms/) through Langchain.

When it comes to selecting open-source language models, there are some rules of thumb to follow, given that the quality of evaluation metrics depends heavily on the model's quality:

1. Opt for models with more than 7 billion parameters. This choice ensures a minimum level of quality in the results for ragas metrics. Models like Llama-2 or Mistral can be an excellent starting point.
2. Always prioritize finetuned models over base models. Finetuned models tend to follow instructions more effectively, which can significantly improve their performance.
3. If your project focuses on a specific domain, such as science or finance, prioritize models that have been pre-trained on a larger volume of tokens from your domain of interest. For instance, if you are working with research data, consider models pre-trained on a substantial number of tokens from platforms like arXiv or Semantic Scholar.

:::{note}
Choosing the right Open-Source LLM for evaluation can by tricky. You can also fine-tune these models to get even better performance on Ragas meterics. If you need some help/advice on that feel free to [talk to us](https://calendly.com/shahules/30min)
:::

In this example we are going to use [vLLM](https://github.com/vllm-project/vllm) for hosting a `HuggingFaceH4/zephyr-7b-alpha`. Checkout the [quickstart](https://vllm.readthedocs.io/en/latest/getting_started/quickstart.html) for more details on how to get started with vLLM.

In [ ]:
!pip3 install "fschat[model_worker,webui]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 23.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 31.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of gradio to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.8/304.8 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 61.5 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [23]:
!curl ipv4.icanhazip.com

34.16.171.228


In [51]:
!pip install litellm[proxy] -Uqq

In [53]:
!litellm --model huggingface/bigcode/starcoder

Traceback (most recent call last):
  File "/usr/local/bin/litellm", line 5, in <module>
    from litellm import run_server
  File "/usr/local/lib/python3.10/dist-packages/litellm/__init__.py", line 419, in <module>
    from .main import *  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/litellm/main.py", line 84, in <module>
    openai_text_completions = OpenAITextCompletion()
  File "/usr/local/lib/python3.10/dist-packages/litellm/llms/openai.py", line 551, in __init__
    self._client_session = self.create_client_session()
  File "/usr/local/lib/python3.10/dist-packages/litellm/llms/base.py", line 12, in create_client_session
    _client_session = httpx.Client()
  File "/usr/local/lib/python3.10/dist-packages/httpx/_client.py", line 686, in __init__
    self._transport = self._init_transport(
  File "/usr/local/lib/python3.10/dist-packages/httpx/_client.py", line 734, in _init_transport
    return HTTPTransport(
  File "/usr/local/lib/python3.10/dist-packages/httpx/_tr

In [5]:
!python -u -m vllm.entrypoints.openai.api_server \
       --host 0.0.0.0 \
       --model TheBloke/Mistral-7B-Instruct-v0.2-AWQ \
       --dtype half \
       --max-num-batched-tokens 4096 \
       --max-model-len 256 \
       --quantization awq \
       --tensor-parallel-size 1 \
       --port 8010   & npx localtunnel --port 8010



^C


Now lets create an Langchain llm instance and wrap it with `RagasLLM` class. Because vLLM can run in OpenAI compatibilitiy mode, we can use the `ChatOpenAI` class as it is with small tweaks.

In [45]:
from langchain.chat_models import ChatOpenAI
from ragas.llms import LangchainLLM

inference_server_url = "http://localhost:8010/v1/generate"

# create vLLM Langchain instance
chat = ChatOpenAI(
    model="TheBloke/Mistral-7B-Instruct-v0.2-AWQ",
    openai_api_key="api-key",
    openai_api_base=inference_server_url,
    max_tokens=256,
    temperature=0.2,
)

# use the Ragas LangchainLLM wrapper to create a RagasLLM instance
vllm = LangchainLLM(llm=chat)

Now lets import all the metrics you want to use and change the llm.

In [46]:
from ragas.metrics import (
    context_precision,
    faithfulness,
    context_recall,
)
from ragas import evaluate
from ragas.metrics.critique import harmfulness

# change the LLM

faithfulness.llm = vllm
context_precision.llm = vllm
context_recall.llm = vllm
harmfulness.llm = vllm
# evaluate


faithfulness.llm

In [47]:
chat

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7dce7e760e20>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7dce7e906b30>, model_name='TheBloke/Mistral-7B-Instruct-v0.2-AWQ', temperature=0.2, openai_api_key='api-key', openai_api_base='http://localhost:8010/v1/generate', openai_proxy='', max_tokens=256)

Now you can run the evaluations with and analyse the results.

In [48]:
# evaluate


result = evaluate(
    fiqa_eval["baseline"].select(range(5)),  # showing only 5 for demonstration
    metrics=[faithfulness],
)

result

evaluating with [faithfulness]


  0%|          | 0/1 [00:02<?, ?it/s]


APIConnectionError: ignored